In [1]:
import tensorflow as tf

In [2]:
import pandas as pd

In [3]:
train_data = pd.read_csv('train.csv')

In [4]:
train_data.shape

(42000, 785)

In [5]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_labels = train_data.label
train_labels.shape

(42000,)

In [7]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
encoder.fit(range(10))
    
train_labels = encoder.transform(train_labels)

In [8]:
train_labels[:5]

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [9]:
train_feats = train_data.drop('label', axis=1)
train_feats = train_feats/255
train_feats.shape

(42000, 784)

In [10]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(train_feats, train_labels, test_size = 0.1)

In [11]:
train_x.shape

(37800, 784)

In [12]:
val_x.shape

(4200, 784)

In [13]:
test_data = pd.read_csv('test.csv')
test_data = test_data/255

In [14]:
test_data.shape

(28000, 784)

In [26]:
sess = tf.InteractiveSession()

In [27]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [28]:
learning_rate = tf.placeholder(tf.float32)

In [29]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [30]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [31]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)



In [32]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [33]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [34]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_pred = tf.nn.softmax(y_conv)

In [35]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [51]:
batchSize = 700
lr = 1e-3
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for j in xrange(50):
      for i in range(0, len(train_x), batchSize):
        batch_x = train_x[i: i+batchSize]
        batch_y = train_y[i: i+batchSize]
        train_step.run(feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.5, learning_rate:lr})
      if j%10==0:
        lr/=3
        
      print('epoch %d test accuracy %g' % (j, accuracy.eval(feed_dict={
          x: val_x, y_: val_y, keep_prob: 1.0})))
    
  test_pred = sess.run(tf.argmax(y_pred, 1), feed_dict={x: test_data, keep_prob: 1.0})

epoch 0 test accuracy 0.951429
epoch 1 test accuracy 0.963571
epoch 2 test accuracy 0.974524
epoch 3 test accuracy 0.979286
epoch 4 test accuracy 0.981429
epoch 5 test accuracy 0.98381
epoch 6 test accuracy 0.984524
epoch 7 test accuracy 0.985714
epoch 8 test accuracy 0.986667
epoch 9 test accuracy 0.985476
epoch 10 test accuracy 0.986429
epoch 11 test accuracy 0.986429
epoch 12 test accuracy 0.986667
epoch 13 test accuracy 0.987143
epoch 14 test accuracy 0.986667
epoch 15 test accuracy 0.987143
epoch 16 test accuracy 0.987143
epoch 17 test accuracy 0.986905
epoch 18 test accuracy 0.987857
epoch 19 test accuracy 0.988095
epoch 20 test accuracy 0.988571
epoch 21 test accuracy 0.988571
epoch 22 test accuracy 0.98881
epoch 23 test accuracy 0.988571
epoch 24 test accuracy 0.988571
epoch 25 test accuracy 0.98881
epoch 26 test accuracy 0.988333
epoch 27 test accuracy 0.988095
epoch 28 test accuracy 0.988571
epoch 29 test accuracy 0.988571
epoch 30 test accuracy 0.988333
epoch 31 test accurac

In [52]:
test_pred.shape

(28000,)

In [53]:
test_pred[:5]

array([2, 0, 9, 9, 3])

In [54]:
import numpy as np
submission = pd.DataFrame(data={'ImageId':(np.arange(test_pred.shape[0])+1), 'Label':test_pred})
submission.to_csv('submission.csv', index=False)
submission.tail()

,ImageId,Label
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
27999,28000,2
